In [128]:
!pip install tensorflow

In [129]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.feature_column import numeric_column
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [130]:
iris = pd.read_csv("https://raw.githubusercontent.com/adivii/datasets/main/Iris.csv")

In [131]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [132]:
iris.shape

(150, 6)

In [133]:
iris.dtypes

Id                 int64
SepalLengthCm    float64
SepalWidthCm     float64
PetalLengthCm    float64
PetalWidthCm     float64
Species           object
dtype: object

In [134]:
iris.iloc[:,1:4] = iris.iloc[:,1:4].astype(np.float32)

<ipython-input-134-80cd8ca9bc1e>:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  iris.iloc[:,1:4] = iris.iloc[:,1:4].astype(np.float32)


In [135]:
iris.dtypes

Id                 int64
SepalLengthCm    float32
SepalWidthCm     float32
PetalLengthCm    float32
PetalWidthCm     float64
Species           object
dtype: object

In [136]:
iris["Species"] = iris["Species"].map({"Iris-setosa":0,"Iris-virginica":1,"Iris-versicolor":2})

In [137]:
from sklearn.model_selection import train_test_split

In [138]:
X_train, X_test, y_train, y_test = train_test_split(iris.iloc[:,1:5], iris["Species"], test_size=0.33, random_state=42)

In [139]:
X_train.shape

(100, 4)

In [140]:
X_test.shape

(50, 4)

In [141]:
columns = iris.columns[1:5]

In [142]:
print(columns)

Index(['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'], dtype='object')


In [143]:
from tensorflow.feature_column import numeric_column

In [144]:
feature_columns = [numeric_column(k) for k in columns]

In [145]:
def input_fn(df,labels):
    feature_cols = {k:tf.constant(df[k].values,shape = [df[k].size,1]) for k in columns}
    label = tf.constant(labels.values, shape = [labels.size,1])
    return feature_cols,label

In [146]:
from tensorflow.feature_column import numeric_column
from tensorflow.estimator import DNNClassifier

In [147]:
classifier = DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[10, 20, 10],
    n_classes=3
)

In [148]:
def input_fn(features, labels=None, training=True, batch_size=32):
    dataset = tf.data.Dataset.from_tensor_slices(({"SepalLength": features}, labels))

    if training:
        dataset = dataset.shuffle(buffer_size=10000)
        dataset = dataset.batch(batch_size)

    return dataset

In [149]:
from tensorflow.keras import layers

In [150]:
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(4,)),
    layers.Dense(32, activation='relu'),
    layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [151]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
4/4 [==============================] - 1s 104ms/step - loss: 1.3125 - accuracy: 0.3400 - val_loss: 1.1402 - val_accuracy: 0.3200
Epoch 2/10
4/4 [==============================] - 0s 21ms/step - loss: 1.1120 - accuracy: 0.3400 - val_loss: 1.0453 - val_accuracy: 0.3200
Epoch 3/10
4/4 [==============================] - 0s 23ms/step - loss: 1.0085 - accuracy: 0.3400 - val_loss: 0.9950 - val_accuracy: 0.3200
Epoch 4/10
4/4 [==============================] - 0s 18ms/step - loss: 0.9564 - accuracy: 0.3400 - val_loss: 0.9224 - val_accuracy: 0.4200
Epoch 5/10
4/4 [==============================] - 0s 22ms/step - loss: 0.8963 - accuracy: 0.6500 - val_loss: 0.8471 - val_accuracy: 0.9600
Epoch 6/10
4/4 [==============================] - 0s 30ms/step - loss: 0.8442 - accuracy: 0.8000 - val_loss: 0.7878 - val_accuracy: 0.7000
Epoch 7/10
4/4 [==============================] - 0s 41ms/step - loss: 0.8039 - accuracy: 0.6600 - val_loss: 0.7463 - val_accuracy: 0.7000
Epoch 8/10
4/4 [==========

In [152]:
evaluation = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {evaluation[1]}')

2/2 [==============================] - 0s 11ms/step - loss: 0.6467 - accuracy: 0.9000
Test Accuracy: 0.8999999761581421
